In [1]:
import neptune
from dotenv import load_dotenv
import plotly.io as pio
import re

# pip install python-dotenv
# Jupyter in VS Code, at least, does not appear to load environment variables from .bashrc
# Workaround, create .env in notebooks directory (gitignored) and place API key
# e.g. NEPTUNE_API_TOKEN="abcdef"
load_dotenv()

True

In [2]:
run = neptune.init_run(project="isears/ptbxlae", with_id="PTBXLAE-29", mode="read-only")
run

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/isears/ptbxlae/e/PTBXLAE-29


In [3]:

def save_reconstruction_to_png(path: str):
    run[path].download(destination='/tmp/neptune.html', progress_bar=False)

    with open('/tmp/neptune.html', 'r') as f:
        raw_text = f.read()


    m = re.search(r'{ *Plotly\.newPlot\(.*\)', raw_text, re.DOTALL)
    start_idx = m[0].index('[')

    number_of_opens = 0

    # Just get data
    for idx in range(start_idx, len(m[0])):
        curr_char = m[0][idx]

        if curr_char == "[":
            number_of_opens += 1
        elif curr_char == "]":
            number_of_opens -= 1

            if number_of_opens == 0:
                end_idx = idx
                break

    plotly_data_raw = m[0][start_idx:end_idx+1]

    fig = pio.from_json(plotly_data_raw)
    return fig.to_image("png")



In [4]:
struct = run.get_structure()
struct['visuals']

collected_imgs = list()

for fname in struct['visuals'].keys():
    if 'example0' in fname:
        collected_imgs.append(save_reconstruction_to_png(f"visuals/{fname}"))
        

In [5]:
import imageio

durations = [150] * len(collected_imgs)
durations[-1] = 500
imageio.mimsave('./neptune.gif', [imageio.imread(c) for c in collected_imgs], duration = durations, loop=0)


/tmp/ipykernel_1518426/492016154.py:5: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.



![TrainingGIF](./neptune.gif "training GIF")